In [92]:
import numpy as np
import pandas as pd
import datetime

In [93]:
header_list=['Date','ID','Event']
df=pd.read_csv('data.txt',names=header_list)
df.Date=df.Date.map(lambda x:x.replace('-','/'))
df

,Date,ID,Event
0,2015/01/01,joe@signifyd.com,PURCHASE
1,2015/02/01,fraudster@fraud.com,FRAUD_REPORT
2,2015/02/03,fraudster@fraud.com,FRAUD_REPORT
3,2015/02/10,joe@signifyd.com,PURCHASE
4,2015/02/14,fraudster@fraud.com,PURCHASE
5,2015/03/15,joe@signifyd.com,PURCHASE
6,2015/05/01,joe@signifyd.com,PURCHASE
7,2015/10/01,joe@signifyd.com,PURCHASE


In [115]:
def str_to_DT(date):
    return datetime.datetime.strptime(date,'%Y/%m/%d') 

In [95]:
df1=df[df.Event=='PURCHASE']
df1

,Date,ID,Event
0,2015/01/01,joe@signifyd.com,PURCHASE
3,2015/02/10,joe@signifyd.com,PURCHASE
4,2015/02/14,fraudster@fraud.com,PURCHASE
5,2015/03/15,joe@signifyd.com,PURCHASE
6,2015/05/01,joe@signifyd.com,PURCHASE
7,2015/10/01,joe@signifyd.com,PURCHASE


In [161]:
status=[]
for i in df1.index:
    ID=df1.loc[i]['ID']
    date=df1.loc[i]['Date']    
    previous=df[(df.ID==ID) & (df.Date<date)]
    
    if len(previous)==0:
        status.append('NO_HISTORY')
        
    elif 'FRAUD_REPORT' in list(previous.Event):
        count=list(previous['Event']).count('FRAUD_REPORT')
        status.append('FRAUD_HISTORY:'+str(count))
    
    elif divmod((str_to_DT(date)-str_to_DT(previous.Date[0])).total_seconds(),86400)[0]<90:
        count=len(previous[previous.Event=='PURCHASE'])
        status.append('UNCONFIRMED_HISTORY:'+str(count))    
    
    else:
        conv=[str_to_DT(d) for d in previous.Date]
        sec_diffs=np.array(str_to_DT(date))-np.array(conv)
        day_diffs=[divmod(duration.total_seconds(),86400)[0] for duration in sec_diffs]
        valid=[1 for x in day_diffs if x>90]
        status.append('GOOD_HISTORY:'+str(len(valid)))

In [163]:
status

['NO_HISTORY',
 'UNCONFIRMED_HISTORY:1',
 'FRAUD_HISTORY:2',
 'UNCONFIRMED_HISTORY:2',
 'GOOD_HISTORY:1',
 'GOOD_HISTORY:4']

In [165]:
df1['Status']=status
df1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Date,ID,Event,Status
0,2015/01/01,joe@signifyd.com,PURCHASE,NO_HISTORY
3,2015/02/10,joe@signifyd.com,PURCHASE,UNCONFIRMED_HISTORY:1
4,2015/02/14,fraudster@fraud.com,PURCHASE,FRAUD_HISTORY:2
5,2015/03/15,joe@signifyd.com,PURCHASE,UNCONFIRMED_HISTORY:2
6,2015/05/01,joe@signifyd.com,PURCHASE,GOOD_HISTORY:1
7,2015/10/01,joe@signifyd.com,PURCHASE,GOOD_HISTORY:4
